In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
# from underthesea import word_tokenize, pos_tag, sent_tokenize
from pyvi.ViTokenizer import tokenize
from pyvi import ViTokenizer
from underthesea import word_tokenize
import warnings
from gensim import corpora, models, similarities
from tqdm import tqdm

import re

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel('data_motobikes.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7208 entries, 0 to 7207
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   7208 non-null   int64 
 1   Tiêu đề              7207 non-null   object
 2   Giá                  7206 non-null   object
 3   Khoảng giá min       7006 non-null   object
 4   Khoảng giá max       7011 non-null   object
 5   Địa chỉ              7167 non-null   object
 6   Mô tả chi tiết       7208 non-null   object
 7   Thương hiệu          7205 non-null   object
 8   Dòng xe              7205 non-null   object
 9   Năm đăng ký          7208 non-null   object
 10  Số Km đã đi          7208 non-null   int64 
 11  Tình trạng           7208 non-null   object
 12  Loại xe              7208 non-null   object
 13  Dung tích xe         7208 non-null   object
 14  Xuất xứ              7208 non-null   object
 15  Chính sách bảo hành  7207 non-null   object
 16  Trọng 

In [4]:
df.columns

Index(['id', 'Tiêu đề', 'Giá', 'Khoảng giá min', 'Khoảng giá max', 'Địa chỉ',
       'Mô tả chi tiết', 'Thương hiệu', 'Dòng xe', 'Năm đăng ký',
       'Số Km đã đi', 'Tình trạng', 'Loại xe', 'Dung tích xe', 'Xuất xứ',
       'Chính sách bảo hành', 'Trọng lượng', 'Href'],
      dtype='object')

In [5]:
data = df[['id', 'Tiêu đề','Mô tả chi tiết']]
data.head()

,id,Tiêu đề,Mô tả chi tiết
0,1,"Bán Vespa Sprint 125cc 2024 xanh dương, xe đẹp...",Bán xe #Vespa Sprint 125cc. Mua mới tại #Topco...
1,2,🔥🔥SH 150i Thắng ABS 2019 BSTP Chính Chủ,"_Bán SH 150i Thắng ABS 2019 Xám Bạc, Úp Team X..."
2,3,CC Vision Thể Thao 2023 Đen+bộ đèn Demi audi A7,Chính chủ bán Vision phiên bản Thể Thao 2023 Đ...
3,4,Vespa Sprint 2019 -125- Đen Đỏ Sport -CHÍNH ...,XE CÁ NHÂN BÁN - XE DO EM ĐỨNG TÊN CHÍNH C...
4,5,Xe tay ga Yamaha Latte 125 – Đăng ký 2021,🛵 Thông tin xe:\n\nDòng xe tay ga cao cấp Yama...


In [6]:
data['Content'] = data['Mô tả chi tiết'].apply(lambda x: ' '.join(x.split()[:200]))
data.head()

,id,Tiêu đề,Mô tả chi tiết,Content
0,1,"Bán Vespa Sprint 125cc 2024 xanh dương, xe đẹp...",Bán xe #Vespa Sprint 125cc. Mua mới tại #Topco...,Bán xe #Vespa Sprint 125cc. Mua mới tại #Topco...
1,2,🔥🔥SH 150i Thắng ABS 2019 BSTP Chính Chủ,"_Bán SH 150i Thắng ABS 2019 Xám Bạc, Úp Team X...","_Bán SH 150i Thắng ABS 2019 Xám Bạc, Úp Team X..."
2,3,CC Vision Thể Thao 2023 Đen+bộ đèn Demi audi A7,Chính chủ bán Vision phiên bản Thể Thao 2023 Đ...,Chính chủ bán Vision phiên bản Thể Thao 2023 Đ...
3,4,Vespa Sprint 2019 -125- Đen Đỏ Sport -CHÍNH ...,XE CÁ NHÂN BÁN - XE DO EM ĐỨNG TÊN CHÍNH C...,XE CÁ NHÂN BÁN - XE DO EM ĐỨNG TÊN CHÍNH C...
4,5,Xe tay ga Yamaha Latte 125 – Đăng ký 2021,🛵 Thông tin xe:\n\nDòng xe tay ga cao cấp Yama...,🛵 Thông tin xe: Dòng xe tay ga cao cấp Yamaha ...


In [7]:
data.columns

Index(['id', 'Tiêu đề', 'Mô tả chi tiết', 'Content'], dtype='object')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7208 entries, 0 to 7207
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              7208 non-null   int64 
 1   Tiêu đề         7207 non-null   object
 2   Mô tả chi tiết  7208 non-null   object
 3   Content         7208 non-null   object
dtypes: int64(1), object(3)
memory usage: 225.4+ KB


In [9]:
stop_word_file = 'files/vietnamese-stopwords.txt'
emojicon_file = 'files/emojicon.txt'
teencode_file = 'files/teencode.txt'


In [10]:
with open(stop_word_file, 'r', encoding='utf-8') as f:
    stopwords = set([w.strip() for w in f.readlines() if w.strip()])

with open(emojicon_file, 'r', encoding='utf-8') as f:
    emojicons = [w.strip() for w in f.readlines() if w.strip()]

with open(teencode_file, 'r', encoding='utf-8') as f:
    teencode_map = {}
    for line in f:
        parts = line.strip().split()
        if len(parts) >= 2:
            teencode_map[parts[0]] = " ".join(parts[1:])


special_tokens = ['', ' ', ',', '.', '...', '-', ':', ';', '?', '%', '(', ')', '+', '/', "'", '&', '#', '*', '!', '"', '_', '=', '[', ']', '{', '}', '~', '`', '|', '\\']


print(f"Stopwords: {len(stopwords)}, Emojis: {len(emojicons)}, Teencode: {len(teencode_map)}")


Stopwords: 1957, Emojis: 67, Teencode: 416


In [11]:
# Các hàm xử lý
def remove_emojis(text):
    for emo in emojicons:
        text = text.replace(emo, ' ')
    return text

def normalize_teencode(text):
    for key, val in teencode_map.items():
        text = re.sub(rf'\b{re.escape(key)}\b', val, text)
    return text

def remove_special_chars(text):
    text = re.sub(r'[^\w\s]', ' ', text)  # loại ký tự đặc biệt
    text = re.sub(r'\s+', ' ', text).strip()  # loại khoảng trắng thừa
    return text

# -----------------------
# 4. TÁCH STOPWORD RIÊNG
# -----------------------
def remove_stopwords(text):
    tokens = word_tokenize(text, format="text").split()
    tokens = [t for t in tokens if t not in stopwords]
    return ' '.join(tokens)

# -----------------------
# 5. CHUẨN HÓA TỔNG HỢP
# -----------------------
def clean_text(text):
    text = str(text).lower()
    text = remove_emojis(text)
    text = normalize_teencode(text)
    text = remove_special_chars(text)
    text = remove_stopwords(text)
    return text

In [12]:
data['clean_text'] = data['Content'].apply(clean_text)
data['clean_text']

0       xe vespa sprint 125 cc mua topcom 01 2024 xe c...
1       _bán sh 150 thắng abs 2019 xám bạc úp team xám...
2       chủ vision phiên_bản thể_thao 2023 đen xe mua ...
3       xe ca_nhân n xe đư chi nh chu_chi tiê t hi_nh ...
4       thông_tin xe dòng xe ga cao_cấp yamaha latte 1...
                              ...                        
7203    xe mua chủ 2024 chạy 1000 km xe bao zin nhu_cầ...
7204    mua xe lead 12 tr 1 giáo giáo_viên thay lốp nồ...
7205    xe yamaha janus 2017 fi smart key màu xanh chủ...
7206    xe cũ đẹp liền lạc xe chất zin xe chủ_kí uy_qu...
7207    đổi xe ducati_scrambler sản_xuất 2019 đk 2020 ...
Name: clean_text, Length: 7208, dtype: object

In [13]:
data.head()

,id,Tiêu đề,Mô tả chi tiết,Content,clean_text
0,1,"Bán Vespa Sprint 125cc 2024 xanh dương, xe đẹp...",Bán xe #Vespa Sprint 125cc. Mua mới tại #Topco...,Bán xe #Vespa Sprint 125cc. Mua mới tại #Topco...,xe vespa sprint 125 cc mua topcom 01 2024 xe c...
1,2,🔥🔥SH 150i Thắng ABS 2019 BSTP Chính Chủ,"_Bán SH 150i Thắng ABS 2019 Xám Bạc, Úp Team X...","_Bán SH 150i Thắng ABS 2019 Xám Bạc, Úp Team X...",_bán sh 150 thắng abs 2019 xám bạc úp team xám...
2,3,CC Vision Thể Thao 2023 Đen+bộ đèn Demi audi A7,Chính chủ bán Vision phiên bản Thể Thao 2023 Đ...,Chính chủ bán Vision phiên bản Thể Thao 2023 Đ...,chủ vision phiên_bản thể_thao 2023 đen xe mua ...
3,4,Vespa Sprint 2019 -125- Đen Đỏ Sport -CHÍNH ...,XE CÁ NHÂN BÁN - XE DO EM ĐỨNG TÊN CHÍNH C...,XE CÁ NHÂN BÁN - XE DO EM ĐỨNG TÊN CHÍNH C...,xe ca_nhân n xe đư chi nh chu_chi tiê t hi_nh ...
4,5,Xe tay ga Yamaha Latte 125 – Đăng ký 2021,🛵 Thông tin xe:\n\nDòng xe tay ga cao cấp Yama...,🛵 Thông tin xe: Dòng xe tay ga cao cấp Yamaha ...,thông_tin xe dòng xe ga cao_cấp yamaha latte 1...


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7208 entries, 0 to 7207
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              7208 non-null   int64 
 1   Tiêu đề         7207 non-null   object
 2   Mô tả chi tiết  7208 non-null   object
 3   Content         7208 non-null   object
 4   clean_text      7208 non-null   object
dtypes: int64(1), object(4)
memory usage: 281.7+ KB


In [15]:
data.columns

Index(['id', 'Tiêu đề', 'Mô tả chi tiết', 'Content', 'clean_text'], dtype='object')

In [16]:
vectorizer = TfidfVectorizer(
    analyzer='word',
    max_features=8000
)
tfidf_matrix = vectorizer.fit_transform(data['clean_text'])
print("Shape TF-IDF:", tfidf_matrix.shape)

Shape TF-IDF: (7208, 8000)


### Consin Simalarity

In [17]:
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
print("Cosine similarity matrix shape:", cosine_sim_matrix.shape)

Cosine similarity matrix shape: (7208, 7208)


In [18]:
data_show = pd.DataFrame(cosine_sim_matrix)
data_show

,0,1,2,3,4,5,6,7,8,9,...,7198,7199,7200,7201,7202,7203,7204,7205,7206,7207
0,1.000000,0.010010,0.042739,0.010077,0.058899,0.062430,0.117835,0.035624,0.051165,0.078843,...,0.054485,0.048460,0.059432,0.063983,0.029400,0.207776,0.057208,0.074375,0.059106,0.004182
1,0.010010,1.000000,0.026529,0.024690,0.014651,0.004013,0.007642,0.029320,0.018933,0.034610,...,0.036796,0.034389,0.086733,0.007490,0.004468,0.033342,0.058151,0.061670,0.022135,0.034215
2,0.042739,0.026529,1.000000,0.015068,0.062520,0.008429,0.016051,0.057184,0.055128,0.058608,...,0.020041,0.031258,0.023835,0.050278,0.022892,0.087660,0.043359,0.026643,0.038182,0.023762
3,0.010077,0.024690,0.015068,1.000000,0.015735,0.007525,0.041835,0.041549,0.019059,0.038008,...,0.008401,0.035247,0.007807,0.014045,0.008379,0.030416,0.007874,0.015101,0.021854,0.006579
4,0.058899,0.014651,0.062520,0.015735,1.000000,0.023428,0.053589,0.033012,0.023118,0.078809,...,0.022728,0.050014,0.026261,0.057024,0.042388,0.056372,0.022769,0.104887,0.047744,0.027955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7203,0.207776,0.033342,0.087660,0.030416,0.056372,0.045687,0.079550,0.074161,0.160331,0.107681,...,0.052548,0.072610,0.040834,0.198860,0.062357,1.000000,0.079355,0.045645,0.109901,0.052784
7204,0.057208,0.058151,0.043359,0.007874,0.022769,0.005360,0.010208,0.007938,0.025290,0.059104,...,0.046551,0.012830,0.119726,0.030616,0.005969,0.079355,1.000000,0.084363,0.034469,0.003268
7205,0.074375,0.061670,0.026643,0.015101,0.104887,0.010280,0.019578,0.040752,0.041015,0.105069,...,0.040970,0.047482,0.138863,0.019188,0.149839,0.045645,0.084363,1.000000,0.053000,0.026978
7206,0.059106,0.022135,0.038182,0.021854,0.047744,0.062623,0.023281,0.037695,0.064465,0.099562,...,0.111308,0.034642,0.012683,0.047505,0.013613,0.109901,0.034469,0.053000,1.000000,0.016634


In [19]:
def recommend(item_id: int, top_n: int = 5):
    """
    Recommend similar motorbikes based on cosine similarity.
    Args:
        item_id (int): id hoặc index của xe trong DataFrame
        top_n (int): số lượng gợi ý muốn lấy
    Returns:
        DataFrame chứa các xe tương tự
    """
    if item_id not in data.index:
        raise ValueError(f"id {item_id} không tồn tại trong DataFrame")

    # Lấy hàng tương ứng trong ma trận cosine
    sim_scores = list(enumerate(cosine_sim_matrix[item_id]))

    # Sắp xếp theo độ tương đồng giảm dần, bỏ chính nó
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1: top_n + 1]

    # Lấy index xe tương tự
    similar_indices = [i[0] for i in sim_scores]
    similar_scores = [i[1] for i in sim_scores]

    # Tạo DataFrame kết quả
    recommendations = data.loc[similar_indices, ['id', 'Tiêu đề', 'Content']].copy()
    recommendations['similarity'] = similar_scores
    return recommendations.reset_index(drop=True)

In [20]:
def recommend_cosine(item_id: int, top_n: int = 5):
    sim_scores = cosine_sim_matrix[item_id]
    sim_scores[item_id] = -1  # bỏ chính nó

    similar_idx = sim_scores.argsort()[::-1][:top_n]
    similar_scores = sim_scores[similar_idx]

    results = data.iloc[similar_idx][['id', 'Tiêu đề', 'Content']].copy()
    results['similarity'] = similar_scores
    return results.reset_index(drop=True)

In [21]:
sample_id = 3
print(f"\nXe gốc:\n{data.loc[sample_id, ['id', 'Tiêu đề', 'Content']]}")
print("\nGợi ý các xe tương tự:")
recommendation = recommend(sample_id, top_n=5)
recommendation


Xe gốc:
id                                                         4
Tiêu đề    Vespa Sprint 2019 -125- Đen Đỏ Sport -CHÍNH ...
Content    XE CÁ NHÂN BÁN - XE DO EM ĐỨNG TÊN CHÍNH C...
Name: 3, dtype: object

Gợi ý các xe tương tự:


,id,Tiêu đề,Content,similarity
0,4116,xe nhà đang xử dụng nên cần bán a,Xe màu đen nhám áo mới làm keo trong Xe dan ba...,0.257452
1,6515,trước mua củ ở cửa hàng xe máy . biển số sg q1 .,Xe ko chính chủ xe ddag ở củ chi . Nay minh đổ...,0.252209
2,3251,Epicupro chính chủ,Xe hoàng hảo điện nước các kiểu ổn địn...,0.247112
3,3172,439 ho hoc lam,Xe bi mất chìa khóa máy im xe như hình bán cho...,0.245392
4,6048,SH 150i ABS ĐEN NHÁM CUỐI 2021 CHÍNH CHỦ.,🌈SH 150i ABS ĐEN NHÁM CUỐI 2021. ➖#BS_80508. ...,0.238655


In [22]:
import joblib

# 2️ Lưu cosine matrix và vectorizer để tái sử dụng
joblib.dump(cosine_sim_matrix, 'cosine_sim_matrix.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
print("Đã lưu cosine_sim_matrix.pkl và tfidf_vectorizer.pkl")


Đã lưu cosine_sim_matrix.pkl và tfidf_vectorizer.pkl


In [23]:
def load_cosine_data():
    """
    Load cosine similarity matrix và vectorizer đã lưu trước đó.
    """
    cosine_matrix = joblib.load('cosine_sim_matrix.pkl')
    vectorizer_loaded = joblib.load('tfidf_vectorizer.pkl')
    print("Đã load cosine matrix & vectorizer thành công.")
    return cosine_matrix, vectorizer_loaded

# Gensim

In [24]:
content_gem = [[text for text in x.split()] for x in data.clean_text]

In [25]:
len(content_gem)

7208

In [26]:
content_gem[:1]

[['xe',
  'vespa',
  'sprint',
  '125',
  'cc',
  'mua',
  'topcom',
  '01',
  '2024',
  'xe',
  'chủ',
  'chạy',
  '14',
  '000',
  'km',
  'giá',
  '66',
  'tr',
  'thương_lượng',
  'anh_chị',
  'nhắn_Zalo',
  'công_chứng',
  'hợp_đồng',
  'mua_bán']]

In [27]:
dictionary = corpora.Dictionary(content_gem)

In [28]:
dictionary.token2id

{'000': 0,
 '01': 1,
 '125': 2,
 '14': 3,
 '2024': 4,
 '66': 5,
 'anh_chị': 6,
 'cc': 7,
 'chạy': 8,
 'chủ': 9,
 'công_chứng': 10,
 'giá': 11,
 'hợp_đồng': 12,
 'km': 13,
 'mua': 14,
 'mua_bán': 15,
 'nhắn_Zalo': 16,
 'sprint': 17,
 'thương_lượng': 18,
 'topcom': 19,
 'tr': 20,
 'vespa': 21,
 'xe': 22,
 '1': 23,
 '150': 24,
 '2019': 25,
 '_bán': 26,
 '_mua': 27,
 '_xe': 28,
 '_xem': 29,
 'abs': 30,
 'biển': 31,
 'bạc': 32,
 'bốc': 33,
 'giấy': 34,
 'keng': 35,
 'ký': 36,
 'kỹ': 37,
 'máy_móc': 38,
 'nguyên': 39,
 'nhà_tân': 40,
 'nhì': 41,
 'nhạc': 42,
 'nốt': 43,
 'p_tân': 44,
 'q': 45,
 'sang_tên': 46,
 'sh': 47,
 'sơn': 48,
 'team': 49,
 'test': 50,
 'thoải_mái': 51,
 'thành_phố': 52,
 'thắng': 53,
 'tân_phú': 54,
 'xi_măng': 55,
 'xám': 56,
 'zin': 57,
 'êm': 58,
 'úp': 59,
 '09': 60,
 '10': 61,
 '2023': 62,
 '3': 63,
 '5': 64,
 '7': 65,
 'a7': 66,
 'a7_giá': 67,
 'audi': 68,
 'bảo_dưỡng': 69,
 'demi': 70,
 'gia_đình': 71,
 'giữ_gìn': 72,
 'hãng': 73,
 'led': 74,
 'liên_hệ': 75,
 '

In [29]:
feature_cnt = len(dictionary.token2id)
feature_cnt

12231

In [30]:
corpus = [dictionary.doc2bow(text) for text in content_gem]

In [31]:
corpus[1]

[(9, 1),
 (22, 2),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 2),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 2),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 2),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 2),
 (57, 1),
 (58, 1),
 (59, 1)]

In [32]:
# Use TF-IDF Model to process corpus, obtaining index
tfidf = models.TfidfModel(corpus)
# tính toán sự tương tự trong ma trận thưa thớt
index = similarities.SparseMatrixSimilarity(tfidf[corpus],
                                            num_features = feature_cnt)
# ma tran: n x n

In [33]:
data_1 = pd.DataFrame(index)
data_1

,0,1,2,3,4,5,6,7,8,9,...,7198,7199,7200,7201,7202,7203,7204,7205,7206,7207
0,1.000000,0.001832,0.017369,0.000165,0.032576,0.040736,0.085422,0.012458,0.015705,0.037873,...,0.028935,0.031461,0.041328,0.031471,0.011843,0.158824,0.026359,0.037390,0.024457,0.000079
1,0.001832,1.000000,0.010755,0.013058,0.006481,0.008879,0.000151,0.017103,0.003490,0.011595,...,0.019793,0.021403,0.066909,0.000121,0.009049,0.010653,0.046150,0.033392,0.006666,0.022845
2,0.017369,0.010755,1.000000,0.001102,0.040386,0.000157,0.000307,0.070246,0.015209,0.017002,...,0.004927,0.012582,0.006267,0.019507,0.006677,0.032327,0.015719,0.003613,0.008858,0.010501
3,0.000165,0.013058,0.001102,1.000000,0.002835,0.005770,0.017066,0.015852,0.000314,0.005768,...,0.000137,0.016721,0.000143,0.000195,0.005880,0.004416,0.007783,0.000233,0.001796,0.000764
4,0.032576,0.006481,0.040386,0.002835,1.000000,0.017124,0.026969,0.008547,0.000441,0.031863,...,0.007130,0.026201,0.012603,0.025244,0.027224,0.019865,0.019893,0.059628,0.014750,0.015008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7203,0.158824,0.010653,0.032327,0.004416,0.019865,0.017760,0.042014,0.029043,0.072953,0.033360,...,0.021127,0.033864,0.013678,0.105504,0.028798,1.000000,0.031037,0.007886,0.043065,0.029803
7204,0.026359,0.046150,0.015719,0.007783,0.019893,0.033485,0.000191,0.000128,0.004413,0.027712,...,0.017817,0.015888,0.088036,0.010867,0.034127,0.031037,1.000000,0.048107,0.009750,0.000054
7205,0.037390,0.033392,0.003613,0.000233,0.059628,0.000203,0.000396,0.011887,0.006278,0.049021,...,0.019168,0.027542,0.101994,0.000319,0.101571,0.007886,0.048107,1.000000,0.018027,0.016921
7206,0.024457,0.006666,0.008858,0.001796,0.014750,0.029891,0.000500,0.008843,0.017129,0.032788,...,0.065674,0.012692,0.000295,0.010614,0.000261,0.043065,0.009750,0.018027,1.000000,0.004622


In [34]:
# giả sử df_1 là ma trận tương đồng n×n, index và columns đều là chỉ số/ID
id = 3
row = data_1.loc[id]

top5 = row.drop(id, errors='ignore').nlargest(5)   # bỏ chính nó
top5_indices = top5.index.tolist()                # <-- lấy index của Series
print(top5_indices)   
data.iloc[top5_indices]                            

[6200, 436, 6849, 295, 296]


,id,Tiêu đề,Mô tả chi tiết,Content,clean_text
6200,6201,Bán xe chính chủ,"Xe chính chủ cần bán honda deam lùn, xe s...","Xe chính chủ cần bán honda deam lùn, xe s...",xe chi nh chu_câ n n honda deam lu n xe sưu_tâ...
436,437,Vario 160 1 đời chủ Ohlis + 4road bs quận 5,Vario 160 indo 1 đời chủ biển số quận 5. ...,Vario 160 indo 1 đời chủ biển số quận 5. ...,vario 160 indo 1 đơ chu_biê n sô quâ n 5 xe 23...
6849,6850,suzuki viva,nhà còn 1 chiếc xe như hình bán cho ai câ...,nhà còn 1 chiếc xe như hình bán cho ai câ...,nha co_n 1 chiê xe hi_nh n câ n la m mai_mo sa...
295,296,Exciter 150 2015 bstp,Exciter 150 2015 bstp giấy tờ đầy đủ không...,Exciter 150 2015 bstp giấy tờ đầy đủ không...,exciter 150 2015 bstp giâ y_tơ đâ y_đu chi nh ...
296,297,Exciter 150 2015 bstp,Exciter 150 2015 bstp giấy tờ đầy đủ không...,Exciter 150 2015 bstp giấy tờ đầy đủ không...,exciter 150 2015 bstp giâ y_tơ đâ y_đu chi nh ...


In [35]:
# Trường hợp khách hàng nhập thông tin tìm kếm
search_str = "xe chính chủ, nguyên kiện"
search_str_wt = clean_text(search_str)
print(search_str_wt.split())
# content_gem_re[:1]

['xe', 'chủ_nguyên', 'kiện']


In [36]:
view_content = search_str_wt.split()
kw_vector = dictionary.doc2bow(view_content)
sim = index[tfidf[kw_vector]]

In [37]:
# sim là numpy array chứa độ tương đồng
# Tạo DataFrame gồm 2 cột: id và sim
df_sim = pd.DataFrame({
    "id": range(len(sim)),
    "sim": sim
})

# Sắp xếp theo sim giảm dần
df_sorted_search = df_sim.sort_values(by="sim", ascending=False)
recommend = df_sorted_search.head()
recommend

,id,sim
3735,3735,0.514529
3449,3449,0.428086
5605,5605,0.291052
4609,4609,0.273028
1501,1501,0.247979


In [38]:
data.iloc[recommend["id"]]

,id,Tiêu đề,Mô tả chi tiết,Content,clean_text
3735,3736,"Bán xe Winner 150 chính chủ,nguyên zin","Bán xe winner 150 chính chủ.nguyên zin, mới le...","Bán xe winner 150 chính chủ.nguyên zin, mới le...",xe winner 150 chủ_nguyên zin leng keng liên_hệ
3449,3450,Cần bán gấp wave chính chủ 110 2023 bstp,Wave 110 đời 2023\nodo 19k\nbiển số tp chính c...,Wave 110 đời 2023 odo 19k biển số tp chính chủ...,wave 110 đời 2023 odo 19 k biển tp chủ_nguyên ...
5605,5606,Future led 8/2022 bstp chính chủ xe ít sử dụng,Future led bstp chính chủ 8/2022 đầy đủ phụ ki...,Future led bstp chính chủ 8/2022 đầy đủ phụ ki...,future led bstp chủ 8 2022 đầy_đủ phụ kiện xe ...
4609,4610,Chính chủ cần bán Exciter 150 2017 máy zin,Bán xe exciter 150 chính chủ nguyên zin\n- Biể...,Bán xe exciter 150 chính chủ nguyên zin - Biển...,xe exciter 150 chủ_nguyên zin biển lâm_đồng 49...
1501,1502,"Cần bán Wave Rs bản 2010 , xe 1 đời chủ nguyên...",Cần bán Wave Rs 110 đời 2010 xe 1 đời chủ nguy...,Cần bán Wave Rs 110 đời 2010 xe 1 đời chủ nguy...,wave rs 110 đời 2010 xe 1 đời chủ_nguyên zin h...


In [39]:
class Recommender:
    def __init__(self, dictionary, tfidf, index, data: pd.DataFrame, clean_text):
        self.dictionary = dictionary
        self.tfidf = tfidf
        self.index = index
        self.data = data
        self.clean_text = clean_text

    def recommend_by_text(self, search_str: str, top_n: int = 5) -> pd.DataFrame:
        """Gợi ý theo nội dung văn bản."""
        search_str_wt = self.clean_text(search_str)
        tokens = search_str_wt.split()
        kw_vector = self.dictionary.doc2bow(tokens)
        sim = self.index[self.tfidf[kw_vector]]
        df_sim = pd.DataFrame({"id": range(len(sim)), "sim": sim}).sort_values(by="sim", ascending=False)
        recommend = df_sim.head(top_n)
        results = self.data.iloc[recommend["id"].to_list()].copy()
        results["sim"] = recommend["sim"].values
        return results

    def recommend_by_id(self, id: int, top_n: int = 5) -> pd.DataFrame:
        """Gợi ý các tài liệu tương tự với doc_id trên ma trận tương đồng n×n."""
        # Lấy dòng tương đồng từ ma trận
        data_1 = pd.DataFrame(self.index)
        try:
            row = data_1.loc[id]
        except KeyError:
            raise ValueError(f"id {id} không tồn tại trong ma trận tương đồng.")

        # Bỏ chính nó, lấy top N
        top_n_sim = row.drop(id, errors="ignore").nlargest(top_n)
        top_n_indices = top_n_sim.index.tolist()

        # Truy xuất dữ liệu tương ứng
        results = self.data.iloc[top_n_indices].copy()
        results["sim"] = top_n_sim.values

        return results

In [40]:
# giả sử df_1 là ma trận tương đồng n×n, index và columns đều là chỉ số/ID
id = 0
row = data_1.loc[id]

top5 = row.drop(id, errors='ignore').nlargest(5)   # bỏ chính nó
top5_indices = top5.index.tolist()                 # <-- lấy index của Series
print(top5_indices)                               # [chỉ số các item tương tự nhất]

[3817, 6730, 4410, 3456, 4726]


In [41]:
rec = Recommender(dictionary, tfidf, index, data, clean_text)
recommend_by_text = rec.recommend_by_text
recommend_by_id = rec.recommend_by_id

In [42]:
search_str = "xe còn mới, máy êm, hao xăng ít, đời từ 2019 trở lên. Nếu có Vision hoặc Janus chạy dưới 10.000km thì càng tốt."
recommend_by_text(search_str)


,id,Tiêu đề,Mô tả chi tiết,Content,clean_text,sim
6604,6605,Xe janus xám đen bảng smartkey,Cần bán xe janus \nDo mua xe mới nên cần bán lại,Cần bán xe janus Do mua xe mới nên cần bán lại,xe janus mua xe,0.447361
6442,6443,Cần bán xe tay ga Yamaha Janus,Xe chính chủ cần bán xe Yamaha Janus ai cần mu...,Xe chính chủ cần bán xe Yamaha Janus ai cần mu...,xe chủ xe yamaha janus mua liên_hệ xe,0.349798
1992,1993,janus,Janus đỏ đen đèn rất sáng giấy tờ đầy đủ,Janus đỏ đen đèn rất sáng giấy tờ đầy đủ,janus đỏ đen đèn giấy_tờ đầy_đủ,0.304264
3123,3124,Bán Janus xe nữ đi đầu đủ giấy tờ,Bán Janus xe nữ đi đầu đủ giấy tờ \nGiá cả thư...,Bán Janus xe nữ đi đầu đủ giấy tờ Giá cả thươn...,janus xe nữ đầu giấy_tờ giá_cả thương_lượng,0.284921
6576,6577,Bán Vision 215 màu đỏ biển tp thì chia lại cho e,Anh chị nào muốn lên đời hoặc nhà dư vision đờ...,Anh chị nào muốn lên đời hoặc nhà dư vision đờ...,anh_chị đời nhà_dư vision đời 215 trở màu đỏ b...,0.269901


In [43]:
id = 3
sample_id = df.loc[id, 'id']
print(f"\nXe gốc:\n{df.loc[df['id'] == sample_id, ['id', 'Tiêu đề', 'Mô tả chi tiết']].iloc[0]}")
print("\nGợi ý các xe tương tự:")
recommend_by_id(id)


Xe gốc:
id                                                                4
Tiêu đề           Vespa Sprint 2019 -125- Đen Đỏ Sport -CHÍNH ...
Mô tả chi tiết    XE CÁ NHÂN BÁN - XE DO EM ĐỨNG TÊN CHÍNH C...
Name: 3, dtype: object

Gợi ý các xe tương tự:


,id,Tiêu đề,Mô tả chi tiết,Content,clean_text,sim
6200,6201,Bán xe chính chủ,"Xe chính chủ cần bán honda deam lùn, xe s...","Xe chính chủ cần bán honda deam lùn, xe s...",xe chi nh chu_câ n n honda deam lu n xe sưu_tâ...,0.331512
436,437,Vario 160 1 đời chủ Ohlis + 4road bs quận 5,Vario 160 indo 1 đời chủ biển số quận 5. ...,Vario 160 indo 1 đời chủ biển số quận 5. ...,vario 160 indo 1 đơ chu_biê n sô quâ n 5 xe 23...,0.329765
6849,6850,suzuki viva,nhà còn 1 chiếc xe như hình bán cho ai câ...,nhà còn 1 chiếc xe như hình bán cho ai câ...,nha co_n 1 chiê xe hi_nh n câ n la m mai_mo sa...,0.309942
295,296,Exciter 150 2015 bstp,Exciter 150 2015 bstp giấy tờ đầy đủ không...,Exciter 150 2015 bstp giấy tờ đầy đủ không...,exciter 150 2015 bstp giâ y_tơ đâ y_đu chi nh ...,0.299775
296,297,Exciter 150 2015 bstp,Exciter 150 2015 bstp giấy tờ đầy đủ không...,Exciter 150 2015 bstp giấy tờ đầy đủ không...,exciter 150 2015 bstp giâ y_tơ đâ y_đu chi nh ...,0.299775


In [44]:
def evaluate_cosine(data, cosine_sim_matrix, sample_size=200):
    indices = np.random.choice(data.index, sample_size, replace=False)

    top1_scores = []
    top3_scores = []
    times = []

    for idx in tqdm(indices, desc="Evaluating Cosine"):
        t0 = time.time()

        sim_scores = cosine_sim_matrix[idx]
        sim_scores[idx] = -1  # loại chính nó

        top3_idx = np.argsort(sim_scores)[-3:][::-1]
        top3_sim = sim_scores[top3_idx]

        t1 = time.time() - t0
        times.append(t1)

        top1_scores.append(top3_sim[0])
        top3_scores.append(np.mean(top3_sim))

    return {
        "Model": "Cosine-Similarity",
        "Avg_Time": np.mean(times),
        "Avg_Top1_Sim": np.mean(top1_scores),
        "Avg_Top3_Sim": np.mean(top3_scores)
    }



In [45]:
def evaluate_gensim(model, data, sample_size=200):
    indices = np.random.choice(len(data), size=sample_size, replace=False)

    top1_scores = []
    top3_scores = []
    times = []

    for idx in tqdm(indices, desc="Evaluating Gensim"):
        t0 = time.time()
        recs = model.recommend_by_id(idx, top_n=3)  # dùng đúng hàm của bạn
        t1 = time.time() - t0
        times.append(t1)

        top1_scores.append(recs.iloc[0]["sim"])
        top3_scores.append(recs["sim"].mean())

    return {
        "Model": "Gensim",
        "Avg_Time": np.mean(times),
        "Avg_Top1_Sim": np.mean(top1_scores),
        "Avg_Top3_Sim": np.mean(top3_scores)
    }


In [46]:
import random
import time

results = []

res_gensim = evaluate_gensim(rec, data, sample_size=10)
results.append(res_gensim)

res_cosine = evaluate_cosine(data, cosine_sim_matrix, sample_size=10)
results.append(res_cosine)

df_results = pd.DataFrame(results)
df_results



Evaluating Cosine: 100%|██████████| 10/10 [00:00<00:00, 1681.56it/s]


,Model,Avg_Time,Avg_Top1_Sim,Avg_Top3_Sim
0,Gensim,21.099047,0.409649,0.320596
1,Cosine-Similarity,0.000595,0.451329,0.412692


In [47]:
def recommend_cosine_by_text(query: str, top_n: int = 5):
    """
    Gợi ý xe máy tương tự dựa trên văn bản người dùng nhập vào.
    
    Args:
        query (str): văn bản tìm kiếm
        top_n (int): số lượng gợi ý
    
    Returns:
        DataFrame: danh sách xe tương tự + độ tương đồng
    """

    # 1. Tiền xử lý query bằng hàm clean_text của bạn
    clean_query = clean_text(query)

    # 2. Vector hóa query
    query_vec = vectorizer.transform([clean_query])

    # 3. Tính độ tương đồng cosine giữa query và toàn bộ item
    sims = cosine_similarity(query_vec, tfidf_matrix).flatten()

    # 4. Lấy top N kết quả cao nhất
    top_idx = sims.argsort()[::-1][:top_n]
    top_scores = sims[top_idx]

    # 5. Trả về DataFrame kết quả
    result = data.iloc[top_idx][['id', 'Tiêu đề', 'Content']].copy()
    result["similarity"] = top_scores

    return result.reset_index(drop=True)


In [48]:
recommend_cosine_by_text("xe còn mới, máy êm, hao xăng ít, đời từ 2019 trở lên. Nếu có Vision hoặc Janus chạy dưới 10.000km thì càng tốt.", top_n=10)


,id,Tiêu đề,Content,similarity
0,6605,Xe janus xám đen bảng smartkey,Cần bán xe janus Do mua xe mới nên cần bán lại,0.405727
1,6427,Cần bán xe như hình một chủ,Xe còn tốt ít đi Một đời chủ Xem xe tại nhà Gi...,0.394077
2,1906,Gia đình cần đổi xe cần bán lại xe janus đời 2016,Gia đình cần đổi xe muốn bán lại xe janus đời ...,0.342975
3,6443,Cần bán xe tay ga Yamaha Janus,Xe chính chủ cần bán xe Yamaha Janus ai cần mu...,0.308978
4,5601,"Bán xe Vision 2018 - chạy 30.000km - Giá: 17,5...",Bán xe Vision 2018 nhà đang sử dụng - chạy 30....,0.291242
5,774,"Cá nhân đổi xe, nên bán - Vespa 2014",Xe còn đi tốt Bán cá nhân Đã qua 1 đời chủ. Ti...,0.280459
6,5978,Bán,Xe chính chủ máy zin chưa rớt Odo ~10k km Xe n...,0.279231
7,1479,cần bán xe sh150i abs mới 99% trắng sx 2019,Mình cần bán xe sh150i 2019 abs màu trắng - xe...,0.261588
8,1993,janus,Janus đỏ đen đèn rất sáng giấy tờ đầy đủ,0.261481
9,6577,Bán Vision 215 màu đỏ biển tp thì chia lại cho e,Anh chị nào muốn lên đời hoặc nhà dư vision đờ...,0.257763
